## Update OiDB metadata using JMMC's CatalogAPI

In [1]:
import pyvo
from a2p2.jmmc import Catalog

prod=False
prod=True

if True:
    oidbMainUrl="http://oidb.jmmc.fr"
    oidbTapUrl="http://tap.jmmc.fr/vollt/tap"
else:    
    oidbMainUrl="http://oidb-beta.jmmc.fr"
    oidbTapUrl="http://tap-preprod.jmmc.fr/vollt/tap"

oidbTap = pyvo.dal.TAPService(oidbTapUrl)
oidbCatalog = Catalog("oidb", prod=prod)

INFO - a2p2.jmmc.catalogs  - 2022-10-05 11:11:04,968 - catalogs.py:36 - Create catalog wrapper to access 'oidb' (production API at https://oidb.jmmc.fr/restxq/catalogs)


### Define helper function to fix content

In [2]:
def fixGranule(granule):
    obs_collection=granule["obs_collection"]
    ref_collection = collectionReference[obs_collection]    
    todo = {}
    for col in ref_collection.keys():
        ref_value     = ref_collection[col]
        granule_value = granule[col]
        if ref_value != granule_value:
            todo[col]=ref_value
    if todo:
        todo["id"]=str(granule["id"])
    return todo
        
def fixRecords(query):
    """ Walk accross granules selected by given query and return a list of dict so we can update the oidb catalog using api.updateRows()"""
    updates=[]
    granules = oidbTap.search(query).to_table()
    for granule in granules:
        update = fixGranule(granule)
        if update:
            updates.append(update)
    if updates:
        return updates


### Fix some known cases

In [3]:
collectionReference={
    #"371c145f-7889-47d5-9c14-5f7889c7d510":{"datapi":"KRAUS"},
    #"c5a5d133-a178-4949-a5d1-33a1781949e5":{"datapi":"SOULAIN"},
    #"03a164ca-3110-446c-a164-ca3110746cdd":{"bib_reference":"2022Natur.602..403G" },
    #"12803c68-6124-40ad-803c-68612470ad3d":{"bib_reference":"2022A&A...665A..32D", "calib_level":3}
}

updates=[] # accumulates json update orders

# Fix case 1 : 
reason="Update reference collections"
for collection_id in collectionReference.keys():
    query = f"SELECT ALL * FROM oidb AS t WHERE obs_collection='{collection_id}'"
    u=fixRecords(query)
    if u:
        updates+=u
        print(f"{len(u)} record(s) to update through catalogAPI for '{reason}'")    
    else:
        print(f"nothing to do for '{reason}'")        

# Fix case 2 : 
reason = "Look for some records that may have bad metadata"
dataPiToFix="Guillaume Mella" # this guy can't be a datapi, he often does hesitate between RA and DEC
query = f"SELECT ALL * FROM oidb AS t WHERE ( t.datapi LIKE '%{dataPiToFix}%' )"
u=fixRecords(query)
if u:
    updates+=u
    print(f"{len(u)} record(s) to update through catalogAPI for '{reason}'")    
else:
    print(f"nothing to do for '{reason}'")   

nothing to do for 'Look for some records that may have bad metadata'


In [4]:
updates

[]

In [5]:
for update in updates:
    pass
# uncomment to apply changes
#    oidbCatalog.updateRows(updates)

# That's all folks !!!